# *오차역전파법*

## 5. 활성화 함수 계층 구현하기

- ReLU와 Sigmoid 계층 구현
___

### 1) ReLU 계층

- ReLU의 계산 그래프는 아래와 같이 표현 가능
![](image/fig 5-18.png)
- 이를 수식으로 나타내면;
$$
y = \left\{
    \begin{array}{ll}
    x \quad (x > 0 )\\
    0 \quad (x \leq 0)
    \end{array}
    \right.
$$
- 미분식을 표현하면;
$$
\frac{\delta y}{\delta x} = \left\{
                            \begin{array}{ll}
                            1 \quad (x > 0 )\\
                            0 \quad (x \leq 0)
                            \end{array}
                            \right.
$$
    와 같이 표현 가능

In [1]:
class Relu:
    def __init__(self):
        self.mask = None
        
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.maks] = 0
        
        return out
    
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

- Relu 클래스는 mask라는 인스턴스 변수를 보유
- mask는 True/False로 구성된 넘파이 배열
- 순전파의 입력(x)의 원소값이 0 이하일 때에는 True, 아닐 경우는 False 반환
- i.e. 아래 예제 참조

In [2]:
x = np.array([[1, -.5], [-2, 3]])
print(x)
print()
mask = (x<=0)
print(mask)
print()
out =  x.copy()
out[mask] = 0
print(out)

[[ 1.  -0.5]
 [-2.   3. ]]

[[False  True]
 [ True False]]

[[1. 0.]
 [0. 3.]]


- ReLU 계층은 전기회로의 '스위치'와 유사 기능 수행
- 전류가 흐르면 스위치를 ON, 아닐 경우 스위치를 OFF
___

### 2) Sigmoid 계층

- 시그모이드 함수는 다음 식을 의미
$$
y = \frac{1}{1+exp(-x)}
$$
- 계산 그래프로 표현하면 아래와 같음
![](image/fig 5-19.png)
- 'exp' 노드는 $y = exp(x)$를, '/' 노드는 $y=\frac{1}{x}$ 계산을 수행

___
- 역전파 흐름을 단계별로 보면 다음과 같음

> #### 1단계
> - '/'노드 즉 $y=\frac{1}{x}$를 미분
> $$\frac{\delta y}{\delta x} = - \frac{1}{x^2}\\ \quad= - y^2$$
> - $-y^2$를 상류값에 곱하여 하류에 전달

<br>
> #### 2단계
> - '+'노드는 값을 여과없이 전달

<br>
> #### 3단계
> - 'exp'노드는 $ y = exp(x)$를 연산하며 미분값은 $exp(x)$임
> $\frac{\delta y}{\delta x} = exp(x)$
> - 위의 예에서는 $exp(-x)$를 곱해 하류로 전달

<br>
> #### 4단계
> - 'x' 노드는 순전파 떄의 값을 서로 바꿔 하류에 전달
> - 본 예에서는 -1을 전달

<br>
- 전 과정을 계산 그래프로 표현하면 아래와 같음
![](image/fig 5-20.png)

- 역전파의 최종 출력값은 $\frac{\delta L}{\delta y}y^2exp(-x)$임
- 이는 순전파의 입력 x와 출력 y만으로 구현 가능
- 노드 내 중간 계산 과정 생략 가능

<br>
- $\frac{\delta L}{\delta y}y^2exp(-x)$는 아래와 같이 정리 가능
$$\frac{\delta L}{\delta y}y^2exp(-x)\quad = \quad\frac{\delta L}{\delta y}\frac{1}{(1+exp(-x))^2}exp(-x)
\quad\\\quad\quad\quad\quad\quad\quad=\quad\frac{\delta L}{\delta y}\frac{1}{1+exp(-x)}\frac{exp(-x)}{1+exp(-x)}
\\=\quad\frac{\delta L}{\delta y}y(1-y)$$
- 이 경우 sigmoid 계층의 역전파는 순전파의 출력($y$)만으로 계산 가능
![](image/fig 5-21.png)
![](image/fig 5-22.png)
___

In [1]:
class Sigmoid:
    def __init__(self):
        self.out = None
    
    
    def forward(self, x):
        self.out = 1/(1+np.exp(-x))
        
        return self.out
    
    
    def backward(self, dout):
        dx = dout * self.out * (1-self.out)
        
        return dx

- 위 식은 순전파 출력값을 'out'에 보관, 이후 역전파 계산 때 사용하는 방식으로 구현